## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:

# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,40.66,75,75,16.11,broken clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,51.46,79,80,16.37,broken clouds
2,2,Barra Patuca,HN,15.8000,-84.2833,81.16,82,86,6.93,overcast clouds
3,3,Nantucket,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky
4,4,Kodiak,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Nantucket,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky
4,4,Kodiak,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky
6,6,Souillac,MU,-20.5167,57.5167,68.38,60,20,6.91,few clouds
8,8,Ponta Do Sol,PT,32.6667,-17.1000,71.71,74,55,6.11,broken clouds
9,9,Margate,GB,51.3813,1.3862,61.30,98,100,11.59,overcast clouds


In [26]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

193


City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

193


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Nantucket,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky
4,4,Kodiak,US,41.2835,-70.0995,64.09,77,1,6.91,clear sky
6,6,Souillac,MU,-20.5167,57.5167,68.38,60,20,6.91,few clouds
8,8,Ponta Do Sol,PT,32.6667,-17.1000,71.71,74,55,6.11,broken clouds
9,9,Margate,GB,51.3813,1.3862,61.30,98,100,11.59,overcast clouds


In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nantucket,US,64.09,clear sky,41.2835,-70.0995,
4,Kodiak,US,64.09,clear sky,41.2835,-70.0995,
6,Souillac,MU,68.38,few clouds,-20.5167,57.5167,
8,Ponta Do Sol,PT,71.71,broken clouds,32.6667,-17.1000,
9,Margate,GB,61.30,overcast clouds,51.3813,1.3862,
11,Yumen,CN,74.17,scattered clouds,40.2833,97.2000,
14,Arraial Do Cabo,BR,71.80,overcast clouds,-22.9661,-42.0278,
26,East London,GB,61.30,overcast clouds,51.3813,1.3862,
30,Muros,ES,62.24,few clouds,42.7762,-9.0603,
35,Tengzhou,CN,74.17,scattered clouds,40.2833,97.2000,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nantucket,US,64.09,clear sky,41.2835,-70.0995,Jared Coffin House
4,Kodiak,US,64.09,clear sky,41.2835,-70.0995,Jared Coffin House
6,Souillac,MU,68.38,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
8,Ponta Do Sol,PT,71.71,broken clouds,32.6667,-17.1000,Hotel do Campo
9,Margate,GB,61.30,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
14,Arraial Do Cabo,BR,71.80,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
26,East London,GB,61.30,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
30,Muros,ES,62.24,few clouds,42.7762,-9.0603,Apartment Av. Castelao
39,Pokhara,NP,73.76,broken clouds,28.2333,83.9833,Butterfly Lodge
40,Barrow,US,64.09,clear sky,41.2835,-70.0995,Jared Coffin House


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))